In [ ]:
import os
import sys
from pathlib import Path

ASL = os.path.join(str(Path.home()), "ASL")
src = os.path.join(str(Path.home()), "ASL", "src")
os.chdir( ASL )
sys.path.append(ASL)
sys.path.append(src)

# STD
import os
import copy

# MISC
import numpy as np
import torch
import imageio
import cv2
from PIL import Image

# matplotlib
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.backends.backend_agg import FigureCanvasAgg
import matplotlib.patches as patches
import pickle
from PIL import Image, ImageDraw
import skimage
from skimage import measure
from skimage.segmentation import mark_boundaries, find_boundaries

from visu.colors import *
from visu.flow_viz import *

In [ ]:
reference_100 = np.array( [0.785, 0.891,0.652,0.885] )

val_res_paths = ["/home/jonfrey/Results/scannet_cl_buffer_filling/2021-07-19T10:36:55_random",         
"/home/jonfrey/Results/scannet_latent/2021-07-20T23:31:09_no_teacher_with_aug",
"/home/jonfrey/Results/scannet_latent/2021-07-20T23:29:48_no_teacher"
]
                
b = "/home/jonfrey/Documents/master_thesis/Report/images/learning_curves_matrixes/"

labels = ["Random","Latent-Aug","Latent-No-Aug"]

val_res_paths = [s+"/val_res.pkl" for s in val_res_paths]

def mat_from_pkl(a):
    with open(a, "rb") as input_file:
        data = pickle.load(input_file)
    res= np.unique( np.array( data[-1] ), return_index =True) 
    idx = res[1]
    mat = np.zeros( (idx.shape[0], idx.shape[0]))
    data = [np.array(d) for d in data]

    el = data[-1].max()+1
    for i in range( el ):
        for j in range( el  ):
            mat[i,j] = data[j][data[-1] == i][-1]
    return mat


def get_metrics( val_res_path,start=0 ):
    mat = mat_from_pkl(val_res_path)
    mat[start:,start:]
    elements = np.ones_like(mat)
    FWT = np.triu(mat, k =1).sum()  / np.triu(elements, k =1).sum() 
    dia = np.diagonal(mat)
    mat_out = np.copy(mat)
    for i in range(mat.shape[1]):
        mat_out[:,i] -= dia[i]
    BWT = np.tril(mat_out, k =-1).sum() / np.tril(elements, k =-1).sum() 
    ACC = np.tril(mat, k =0).sum()  / np.tril(elements, k =0).sum()
    
    return { "ACC": ACC,
     "FWT":FWT,
     "BWT":BWT,
    "Omega": np.mean( mat[-1,1:]/reference_100 )
    }

metrices = [ get_metrics(v, start=1) for v in val_res_paths ]

for j, m in enumerate( metrices):
    print(labels[j] ,"&", np.round( m['ACC']*100,1) ,"&" , np.round( m['FWT']*100,1) ,"&" , np.round( m['BWT']*100,1) ,"&", np.round( m['Omega']*100,1) , "\\\\")

FWT = [ a["FWT"] for a in metrices ]
BWT = [ a["BWT"] for a in metrices ]
ACC = [ a["ACC"] for a in metrices ]
Omega = [ a["Omega"] for a in metrices ]

metrics = {"ACC": ACC, "FWT": FWT, "BWT": BWT, "Omega": Omega}
metrics["Omega"]

metrics.keys()

In [ ]:
textwidth = 10.7 #cm
fig, ax = plt.subplots(1, 4, figsize=(textwidth, 2.9), sharex=False, sharey=False)
left  = 0.05  # the left side of the subplots of the figure
right = 0.95    # the right side of the subplots of the figure
bottom = 0.4   # the bottom of the subplots of the figure
top = 0.85      # the top of the subplots of the figure
wspace = 0.5   # the amount of width reserved for blank space between subplots
hspace = 0.2  # the amount of height reserved for white space between subplots

plt.subplots_adjust(left=left, bottom=bottom, right=right, top=top, wspace=wspace, hspace=hspace)

#plt.setp(ax, yticks=[1, 2, 3])

for i,key in enumerate( metrics.keys()):
    
    bars = ax[i].bar(labels, metrics[key], facecolor=COL_MAP(i+1),width =0.6)
    #ax[i].set_xlabel(key,fontsize=20, fontname= 'Liberation Serif' )
    # ax[i].set_title(key,fontsize=20, fontname= 'Liberation Serif' )
    #ax[i].set_ylabel(key,fontsize=15, fontname= 'Liberation Serif')
    ax[i].set_title(key,fontsize=20, fontname= 'Liberation Serif', pad=12)
    ax[i].grid(True)
    ax[i].tick_params(axis="x", labelsize=15,rotation=30,width =15,color=(181/255, 181/255, 181/255, 0.36))
    
    
    ax[i].tick_params(axis="y", labelsize=12)

    # Axis formatting.
    ax[i].spines['top'].set_visible(False)
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['left'].set_visible(False)
    ax[i].spines['bottom'].set_color('#DDDDDD')
    
    ax[i].yaxis.grid(True, color='#EEEEEE')
    ax[i].xaxis.grid(False)
    #ax[i].tick_params(bottom=False, left=False)
    ax[i].set_axisbelow(True)
    # Add text annotations to the top of the bars.

    
    if  metrics[key][j] > 0:
        ax[i].set_ylim(0,  max( metrics[key])*1.2  )
    else:
        ax[i].set_ylim(min( metrics[key])*1.2, 0  )
        
    ax[i].set_xlim(    ax[i].get_xlim()[0]-0.5 ,ax[i].get_xlim()[1]+0.5)
    bar_color = bars[0].get_facecolor()
    if metrics[key][0] < 0:
        ax[i].invert_yaxis()
    for j,bar in enumerate(bars):
      h = [ bar.get_height() for bar in bars]
      if  metrics[key][j] > 0:
        bh = bar.get_height() + 0.05
        va = "bottom"
      else:
        bh = bar.get_height() - 0.015
        va = "top"
      ax[i].text(
          bar.get_x() + bar.get_width() / 2,
          bh ,
          round( metrics[key][j]  , 2),ha="center", va=va,
                color=bar_color,
      weight='bold'
      )

        
plt.show()
from visu.visualizer import get_img_from_fig
res = get_img_from_fig(fig, dpi=360)
imageio.imwrite( 'docs/bar_plot_latent.png', res)
# plt.close()
# return np.uint8(arr)